In [3]:
import core_periphery_functions as cpf # import some helpers functions
from multiprocessing import Pool, cpu_count
import pandas as pd
from core_periphery_sbm import core_periphery as cp
from core_periphery_sbm import network_helper as nh
from core_periphery_sbm import model_fit as mf
import networkx as nx
import os

`from core_periphery_sbm `

For calculating core-periphery structure we use code available here https://github.com/ryanjgallagher/core_periphery_sbm 

We constructed 30 days subnetworks [t, t+30] from iteractions data `../data/interactions/`, and for each subnetwork inferred core-periphery structure. For each community we store all subnetworks configurations in hdf5 file `../data/core periphery/community_qa_comm.hdf5`. For example key `labels_0-30days` is list of tupples `(node, group)`. If node is in the core `group=0.0`, otherwise it is `1.0` 

In this example we show how to calculate core-periphery structure of subnetworks using biSBM model. 

Note that our main results our obtained from the sample. We run 50 iterations for each submetwork and choose the best configuration according minimum description lenght. 

In [ ]:
def run_sbm(G):

    hubspoke = cp.HubSpokeCorePeriphery(n_gibbs=100, n_mcmc=10*len(G))
    hubspoke.infer(G)
    labels = hubspoke.get_labels(last_n_samples=50, prob=False, return_dict=True)
    
    #model selection
    inf_labels_hs = hubspoke.get_labels(last_n_samples=50, prob=False, return_dict=False)
    mdl_hubspoke = mf.mdl_hubspoke(G, inf_labels_hs, n_samples=100000)

    return labels, mdl_hubspoke

def mpi_run(edges):
    G = nx.Graph()
    G.add_edges_from(edges)  
    labels, mdl = run_sbm(G)
    ns, ms, Ms = nh.get_block_stats(G, labels, n_blocks=2)
    return labels, ns, ms, Ms, mdl


if __name__ == '__main__':

    name = "astronomy" #"052012astronomy", "052012economics", "052012-literature", "052012-theoretical-physics", "astronomy", "economics", "literature", "physics"
        
    nproc = 2
    print(name)

    tip = 'qa_comm'
    nodes = 'all'             

    loc='../data/interactions/%s/'%name
    qa = pd.read_csv(loc+'%s_interactions_questions_answers.csv'%(name))
    acc = pd.read_csv(loc+'%s_interactions_acc_answers.csv'%(name))
    comm = pd.read_csv(loc+'%s_interactions_comments.csv'%(name))

    window = 30
    data = []

    for i in range (0, 150):
        ltlim = time0 + i
        htlim = time1 + i
        interactions = cpf.merge_interactions(qa, acc, comm, ltlim, htlim, tip=tip)
        edges = cpf.filter_edges(interactions)
        data.append(edges)

    pool = Pool(nproc)
    results = pool.map(mpi_run, data)
    
    path='../data/core_periphery_1itteration/'
    os.makedirs(path, exist_ok=True)
    fname = path+'%s_window%s.hdf5'%(name, window)
    cpf.write_results( results, fname, window)
